In [5]:
# Import packages
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd

# Create driver object and get to imdb homepage
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

driver.get('https://edition.cnn.com/election/2024/primaries-and-caucuses/results/alabama')

List for state.

In [2]:
# Generate the list of each party
super_tuesday_dem = ['Alabama','American Samoa','Arkansas','California','Colorado','Iowa','Maine','Massachusetts',
                     'Minnesota', 'North Carolina', 'Oklahoma', 'Tennessee','Texas', 'Utah', 'Vermont', 'Virginia']


super_tuesday_rep = ['Alabama', 'Alaska', 'Arkansas', 'California', 'Colorado', 'Maine', 'Massachusetts',
                    'Minnesota', 'North Carolina', 'Oklahoma', 'Tennessee', 'Texas', 'Utah', 'Vermont',
                     'Virginia'] 

super_tuesday_dem = [state.lower().replace(' ', '-') for state in super_tuesday_dem]
super_tuesday_rep = [state.lower().replace(' ', '-') for state in super_tuesday_rep]

# Combine the lists of states into one list
merged_super_tuesday = super_tuesday_dem + [s for s in super_tuesday_rep if s not in super_tuesday_dem]

# Output the lists to verify their correctness
print(super_tuesday_dem)
print(super_tuesday_rep)
print(merged_super_tuesday)

['alabama', 'american-samoa', 'arkansas', 'california', 'colorado', 'iowa', 'maine', 'massachusetts', 'minnesota', 'north-carolina', 'oklahoma', 'tennessee', 'texas', 'utah', 'vermont', 'virginia']
['alabama', 'alaska', 'arkansas', 'california', 'colorado', 'maine', 'massachusetts', 'minnesota', 'north-carolina', 'oklahoma', 'tennessee', 'texas', 'utah', 'vermont', 'virginia']
['alabama', 'american-samoa', 'arkansas', 'california', 'colorado', 'iowa', 'maine', 'massachusetts', 'minnesota', 'north-carolina', 'oklahoma', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'alaska']


In [3]:
# Define a function to do the web scraping 
def funct_normal(driver):
    info = []
    data_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find from the class of core-result
    for dem in data_soup.select('article.core-result'):
        County = dem.select_one("h2.header-2-AOgLYo").get_text(strip=True)
        
        # Scrap information from each candidates
        candidates_table = dem.find('table', class_='cnn-pcl-1me6450')
        candidates_rows = candidates_table.find_all('tr')
        candidate_name = candidates_rows[1].select_one("span.name")
        if candidate_name is not None:
            counter = 1
            while counter < len(candidates_rows):
                State = state
                candidate_name = candidates_rows[counter].select_one("span.name")
                if candidate_name is not None:
                    
                    # Find the Candidate, Party, Incumbent, Vote, Pct, Winner
                    Candidate = candidate_name.get_text(strip=True)
                    Party = data_soup.select('h2.header-2-AOgLYo.cnn-pcl-xk8c6r')[0].text.split(' ')[0]
                    Incumbent = 'Yes' if 'Incumbent' in candidates_rows[counter].select_one('span.party-label-239xt1').get_text() else 'No'
                    Vote = candidates_rows[counter].select_one('span[data-testid="votes"]').get_text(strip=True)
                    Pct = candidates_rows[counter].select_one("td.col-vote-percent-1JXrHP").get_text(strip=True)
                    Winner = 'Yes' if data_soup.find('span', {'class': 'name'}).get_text(strip = True) == Candidate else "No"
                    
                    # Find the Delegates
                    candidate_delegates_table = data_soup.find('table', class_='cnn-pcl-1me6450 with-delegates-22I-Fo')
                    candidates_delegates_rows = candidate_delegates_table.find_all('tr')
                    Delegates_element = candidates_delegates_rows[counter].select_one("td.col-delegates-2rW54a[data-testid='delegates']")
                    Delegates = Delegates_element.get_text(strip=True) if Delegates_element else 0
                    Delegates = int(Delegates) if Delegates.isdigit() else 0
                    
                    # Store all candidate information in a dictionary and append to the list
                    info.append({'State': State, 'County': County, 'Candidate': Candidate, 'Party': Party,
                            'Incumbent': Incumbent, 'Vote': Vote, 'Pct': Pct, 'Winner': Winner, 'Delegates': Delegates})
                counter += 1
    return info    

In [6]:
# Initiate an empty list 
all_data =[]

# Search from the merged_super_tuesday
for state in merged_super_tuesday:
    
    # Scrap from Dem
    if state in super_tuesday_dem:
        
        # Excluding the 2 special cases
        if state not in ['alaska', 'american-samoa']:
            driver.get(f'https://edition.cnn.com/election/2024/primaries-and-caucuses/results/{state}/democratic-presidential-primary')
            
            # Click the county button
            WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CLASS_NAME, "cta-wrapper-1KjYeR")))
            county_button = driver.find_element(By.CLASS_NAME, "cta-wrapper-1KjYeR").find_element(By.TAG_NAME, "a")

            while True:
                
                # Use the function to extend the web information 
                all_data.extend(funct_normal(driver))
                try:
                    
                    # Click the rightButton
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "rightButton"))).click()
                except:
                    break
                    
        else:
            
            # Special cases no need to click the 'county' button
            driver.get(f'https://edition.cnn.com/election/2024/primaries-and-caucuses/results/{state}/democratic-presidential-primary')
            all_data.extend(funct_normal(driver))
            
    # Similar as Dem case                
    if state in super_tuesday_rep:
        if state not in ['alaska', 'american-samoa']:
            driver.get(f'https://edition.cnn.com/election/2024/primaries-and-caucuses/results/{state}/republican-presidential-primary')
            WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CLASS_NAME, "cta-wrapper-1KjYeR")))
            county_button = driver.find_element(By.CLASS_NAME, "cta-wrapper-1KjYeR").find_element(By.TAG_NAME, "a")

            while True:
                all_data.extend(funct_normal(driver))
                try:
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "rightButton"))).click()
                except:
                    break
        else:
            driver.get(f'https://edition.cnn.com/election/2024/primaries-and-caucuses/results/{state}/republican-presidential-primary')
            all_data.extend(funct_normal(driver))

# Display the dataframe            
all_df = pd.DataFrame(all_data, columns = ["State", "County", "Candidate", "Party", 'Incumbent', 'Vote', 'Pct', 'Winner', 'Delegates'])
all_df

,State,County,Candidate,Party,Incumbent,Vote,Pct,Winner,Delegates
0,alabama,Autauga,Joe Biden,Democratic,Yes,"1,315",88.6%,Yes,52
1,alabama,Autauga,Dean Phillips,Democratic,No,98,6.6%,No,0
2,alabama,Autauga,Uncommitted,Democratic,No,72,4.8%,No,0
3,alabama,Baldwin,Joe Biden,Democratic,Yes,"2,584",87.6%,Yes,52
4,alabama,Baldwin,Uncommitted,Democratic,No,206,7.0%,No,0
...,...,...,...,...,...,...,...,...,...
10955,virginia,Wythe,Nikki Haley,Republican,No,342,11.8%,No,6
10956,virginia,York,Donald Trump,Republican,No,"5,179",60.7%,Yes,42
10957,virginia,York,Nikki Haley,Republican,No,"3,181",37.3%,No,6
10958,alaska,Republican Presidential Primary: Alaska,Donald Trump,Republican,No,"9,243",87.6%,Yes,29


In [5]:
# Create a CSV file.
df_whole_data = pd.DataFrame(all_df)
df_whole_data.to_csv('group_10——mine', index = False)

In [1]:
# Group members
# Wong Wai Yan
# Vanessa Tsang
# Tan Wai Yin
# Kei So Wan